Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.

Each training and test example is assigned to one of the following labels:

Label	Description
0	T-shirt/top
1	Trouser
2	Pullover
3	Dress
4	Coat
5	Sandal
6	Shirt
7	Sneaker
8	Bag
9	Ankle boot

Class counts are all 1,000:
{0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}



In [ ]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

# You cannot modify from here until it is indicated by a comment
(test_data),test_data_info=tfds.load('fashion_mnist',split='test',with_info=True,as_supervised=True)

(train_data),ds_info=tfds.load('fashion_mnist',split=['train[3000:57000]'],with_info=True,as_supervised=True)



def getnewtst():
  (new_test),new_test_info=tfds.load('fashion_mnist',split=['train[0:2999]'],with_info=True,as_supervised=True)
  new_test = new_test[0].map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
  new_test = new_test.batch(64)
  new_test = new_test.cache()
  new_test = new_test.prefetch(tf.data.AUTOTUNE)
  return new_test


# Can modify code now below this comment Though be careful if you change the
# normalization

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`.
  The model wants the float and tfds gives you 0-255."""
  return tf.cast(image, tf.float32) / 255., label


# The steps below or a variation are required to actually extract the images
# and labels for use in training/testing

train_data = train_data[0].map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.cache()
train_data = train_data.shuffle(ds_info.splits['train'].num_examples)
train_data = train_data.batch(64)
train_data = train_data.prefetch(tf.data.AUTOTUNE)


test_data = test_data.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.batch(128)
test_data = test_data.cache()
test_data = test_data.prefetch(tf.data.AUTOTUNE)



In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu'
                 ),
tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer=tf.keras.optimizers.SGD(momentum=0.3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    train_data,
    epochs=3,
    validation_data=getnewtst(),
)

In [ ]:
'''
# This code checks the count per class and each training has 60K
# Each class in test also has 1K for a total of 10K
def checkclasscnt(name,mysplit):
  ds,ds_info = tfds.load(name,split=mysplit,with_info=True)
  print(ds_info.features["label"].names)
# Get the number of classes
  num_classes = ds_info.features['label'].num_classes

# Create a dictionary to store class counts
  class_counts = {i: 0 for i in range(num_classes)}


  for example in ds:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`

   label = example["label"]
   class_counts[label.numpy()] += 1
  return(class_counts)


ccount=checkclasscnt('fashion_mnist','train')
print(ccount)
'''